In [1]:
from torch.utils.data import DataLoader
import preprocess as pr
import model as md
import torch
from torch import nn, optim
import pickle
from pathlib import Path

In [2]:
# Define Processor
print("Define Processor")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fetch Datasets
print("Fetch Datasets")
data_path = './data/AmazonReviewFull/amazon_review_full_csv/'
df_train = pr.load_data(data_path + "train.csv")
df_test = pr.load_data(data_path + "test.csv")

# Define Model
print("Define Model")
MAX_SEQ_LEN = 32
emb_dim = 300
model = md.Classifier(MAX_SEQ_LEN, emb_dim, 16, 16)
model.to(device)

# Parameters
print("Parameters")
lr = 0.003
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 30
printing_gap = 1
model_path = 'data/best_model.pt'
saved_model_device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 16


Define Processor
Fetch Datasets
Define Model
Parameters


In [3]:
# Loaders
print("TrainData")

df_train, df_test = df_train.iloc[:20], df_test.iloc[:20]

dataset_train = pr.TrainData(df_train, max_seq_len=32)
dataset_test = pr.TrainData(df_test, max_seq_len=32)


TrainData


In [ ]:
print("Collate")
train_collate = lambda batch: pr.collate(batch, vectorizer=dataset_train.vectorizer)
test_collate = lambda batch: pr.collate(batch, vectorizer=dataset_test.vectorizer)

In [ ]:
print("DataLoader")
train_loader = DataLoader(dataset_train, batch_size=batch_size, collate_fn=train_collate)
test_loader = DataLoader(dataset_test, batch_size=batch_size, collate_fn=test_collate)

In [ ]:
def getLoader(loader):
    global result
    result = loader
    return loader



In [9]:


PIK = "data/pickle.dat"

my_file = Path(PIK)
if my_file.is_file():
    with open(PIK, "rb") as f:
        data = pickle.load(f)
else:
    dataset_train = pr.TrainData(df_train, max_seq_len=32)
    dataset_test = pr.TrainData(df_test, max_seq_len=32)
    data = {'dataset_train':dataset_train, 'dataset_test':dataset_test}

    with open(PIK, "wb") as f:
        pickle.dump(data, f)

dataset_train = data['dataset_train']
dataset_test = data['dataset_test']

In [ ]:
# Train Model
printing_gap = 1
epochs = 30
print("Train loop")
pr.train_loop(model, epochs, optimizer, criterion, train_loader, test_loader, emb_dim,
              printing_gap, saved_model_device, model_path, device)
